# AVI-DYS Post Processing

In [24]:
# Import Libraries
import os
import shutil
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import preprocessing
import cv2

## User Specific Parameters (CHANGE THESE)

In [25]:
# Define Parameters
project_path = "C:\\Users\\zacha\\Repos\\AVI-DYS\\AVI-DYS-lower-limb-zach_roth-2023-04-19"
save_path = "C:\\Users\\zacha\\Data\\AVI-DYS\\Results\\Post-Processing"

IDs = ['152']

pcutoff = 0.8

visualizations = True

## Functions

In [26]:
def fill_gaps(df):
    """Fill gaps with sklearn's iterative imputer.

    Args:
        df (DataFrame): raw data with missing values

    Returns:
        DataFrame: data with imputed values
    """
    imp = IterativeImputer(max_iter=10, random_state=42,sample_posterior=True)
    cols = df.columns
    imp.fit(df)
    data = imp.transform(df)
    df = pd.DataFrame(data,columns=cols)
    return df

In [27]:
# Create Lists of Common Variables

# Create multindexes for column names
bodyparts = ['RKNE', 'RANK', 'RD3M', 'RD1M', 'RHLX', 
             'LHLX', 'LD1M', 'LD3M', 'LANK', 'LKNE']
iterables = [bodyparts, ['x','y','likelihood']]
mvmt_cols = pd.MultiIndex.from_product(iterables, names=["bodyparts", "coords"])

segments = ['RKNE_RANK', 'RANK_RD3M', 'RANK_RD1M', 'RD1M_RHLX',
            'LKNE_LANK', 'LANK_LD3M', 'LANK_LD1M', 'LD1M_LHLX']
iterables = [segments, ['length','orientation','likelihood']]
skltn_cols = pd.MultiIndex.from_product(iterables, names=["segments", "coords"])

# Create list of bodyparts per side
left_bodyparts = ['LHLX', 'LD1M', 'LD3M', 'LANK', 'LKNE']
right_bodyparts = ['RKNE', 'RANK', 'RD3M', 'RD1M', 'RHLX']

left_segments = ['LKNE_LANK', 'LANK_LD3M', 'LANK_LD1M', 'LD1M_LHLX']
right_segments = ['RKNE_RANK', 'RANK_RD3M', 'RANK_RD1M', 'RD1M_RHLX']

In [28]:
# Create the save paths
save_raw = os.path.join(save_path,'Raw')
save_imp = os.path.join(save_path,'Imputed')

# Create the data path (the video folder in the project path)
data_path = os.path.join(project_path,'videos')

# Get a list of file names
data_file_names = os.listdir(data_path)

In [29]:
# Process the data

# Iterate over the participants
for i in IDs:
    # Make the Save Folders
    
    # Get the paths to the left and right mvmt and skltn data
    for n in data_file_names:
        if i+'-11' in n and 'filtered.csv' in n:
            left_mvmt_path = os.path.join(data_path,n)
        elif  i+'-12' in n and 'filtered.csv' in n:
            right_mvmt_path = os.path.join(data_path,n)
        elif i+'-11' in n and 'filtered_skeleton.csv' in n:
            left_skltn_path = os.path.join(data_path,n)
        elif i+'-12' in n and 'filtered_skeleton.csv' in n:
            right_skltn_path = os.path.join(data_path,n)
            
    # Read in the movement and skeleton data
    mvmt_left = pd.read_csv(left_mvmt_path,index_col=0,names=mvmt_cols,skiprows=3)
    mvmt_right = pd.read_csv(right_mvmt_path,index_col=0,names=mvmt_cols,skiprows=3)
    skltn_left = pd.read_csv(left_skltn_path,index_col=0,names=skltn_cols,skiprows=2)
    skltn_right = pd.read_csv(right_skltn_path,index_col=0,names=skltn_cols,skiprows=2)

    # Rename columns
    mvmt_left.columns = ['_'.join(col) for col in mvmt_left.columns]
    mvmt_right.columns = ['_'.join(col) for col in mvmt_right.columns]
    skltn_left.columns = ['_'.join(col) for col in skltn_left.columns]
    skltn_right.columns = ['_'.join(col) for col in skltn_right.columns]
    
    # Apply p-cutoff and drop unused columns
    
    # left movement data
    for b in bodyparts:
        likelihood_col = b+'_likelihood'
        x_col = b+'_x'
        y_col=b+'_y'
        if b in right_bodyparts:
            mvmt_left = mvmt_left.drop([likelihood_col,x_col,y_col],axis=1)
        else:
            mvmt_left.loc[mvmt_left[likelihood_col] < 0.8, [x_col, y_col]] = np.NaN
            mvmt_left = mvmt_left.drop([likelihood_col],axis=1)

    # right movement data
    for b in bodyparts:
        likelihood_col = b+'_likelihood'
        x_col = b+'_x'
        y_col=b+'_y'
        if b in left_bodyparts:
            mvmt_right = mvmt_right.drop([likelihood_col,x_col,y_col],axis=1)
        else:
            mvmt_right.loc[mvmt_right[likelihood_col] < 0.8, [x_col, y_col]] = np.NaN
            mvmt_right = mvmt_right.drop([likelihood_col],axis=1)
            
    # left skeleton data
    for b in segments:
        likelihood_col = b+'_likelihood'
        length_col = b+'_length'
        orientation_col=b+'_orientation'
        if b in right_segments:
            skltn_left = skltn_left.drop([likelihood_col,length_col,orientation_col],axis=1)
        else:
            skltn_left.loc[skltn_left[likelihood_col] < 0.8, [length_col, orientation_col]] = np.NaN
            skltn_left = skltn_left.drop([likelihood_col],axis=1)
             
    # right skeleton data
    for b in segments:
        likelihood_col = b+'_likelihood'
        length_col = b+'_length'
        orientation_col=b+'_orientation'
        if b in left_segments:
            skltn_right = skltn_right.drop([likelihood_col,length_col,orientation_col],axis=1)
        else:
            skltn_right.loc[skltn_right[likelihood_col] < 0.8, [length_col, orientation_col]] = np.NaN
            skltn_right = skltn_right.drop([likelihood_col],axis=1)
    
    # Get Scale Factors for Spatial Normalization
    scale_factor_left = max(skltn_left['LKNE_LANK_length'])
    scale_factor_right = max(skltn_right['RKNE_RANK_length'])

    # Save the raw data
    mvmt_left.to_csv(os.path.join(save_raw,f'{i}-mvmt-left.csv'))
    mvmt_right.to_csv(os.path.join(save_raw,f'{i}-mvmt-right.csv'))
    skltn_left.to_csv(os.path.join(save_raw,f'{i}-skltn_left.csv'))
    skltn_right.to_csv(os.path.join(save_raw,f'{i}-skltn_right.csv'))

    # Interpolatate Missing Data with IterativeImputer
    mvmt_left = fill_gaps(mvmt_left)
    mvmt_right = fill_gaps(mvmt_right)
    skltn_left = fill_gaps(skltn_left)
    skltn_right = fill_gaps(skltn_right)

    # Save the raw data
    mvmt_left.to_csv(os.path.join(save_imp,f'{i}-mvmt-left.csv'))
    mvmt_right.to_csv(os.path.join(save_imp,f'{i}-mvmt-right.csv'))
    skltn_left.to_csv(os.path.join(save_imp,f'{i}-skltn_left.csv'))
    skltn_right.to_csv(os.path.join(save_imp,f'{i}-skltn_right.csv'))

